# Natural Language Processing
NLP's are a discipline in Data Science that deals with the communiaits between natural languages and computer langauges. Ac ommon example of an NLP is spellcheck or autocompelte.

### Recurrent Nerual Networks
Algorithm that is much more capable of processing sequential data such as text or characters called a recurrent neural network(RNN).
* Sentiment Analysis
* Character Generation 

### Our Data - Sequential
This data we will look at is sequential, since we will encode our text data

In [ ]:
vocab = {} # map words to integer types
word_encoding = 1 
def bag_of_words(text):
    global word_encoding

    words = text.lower().split(" ")
    bag = {}
    for word in words:
        if word in vocab:
            encoding = vocab[word]
        else:
            vocab[word] = word_encoding
            encoding = word_encoding
            word_encoding += 1
        if encoding in bag:
            bag[encoding] += 1
        else:
            bag[encoding] = 1
    return bag
text = "This is a test to see if this test will work is is test a this that then"
bag = bag_of_words(text)
print(bag)
print(vocab)

{1: 3, 2: 3, 3: 2, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9, 'that': 10, 'then': 11}


The above approach of encoding our data to be numerical values("Bag of Words") will have not perform well. We want our encoding to have vectors that represent very similar words to be very close to one another. In other words the angle between two vectors are small if the words are very similar. This method is called _Word Embedding_.

In [13]:
%tensorflow_version 2.x 
from keras.datasets import imdb
# from keras.preprocessing import sequence
from keras.utils import pad_sequences
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
train_data[0]

We need our data length inputs to our neural network to be the same length. This issue can be solved as such:
* If the review is > 250 words trim off extra words
* If the review is < 250 words add trailing zeros to equal 250 words
keras has a neat function to do this.

In [14]:
train_data = pad_sequences(train_data, MAXLEN)
test_data = pad_sequences(test_data, MAXLEN)

## Building the Model
Let us now create the model. We'll use a word embedding layer and add an LSTM layer that feeds into a Dense node to get our predicted sentiment. 

In [16]:
tf.keras.layers.LSTM

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1,activation="sigmoid") # Squash values b/w [0,1]
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


## Training

In [17]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])
history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 72s 111ms/step - loss: 0.4049 - acc: 0.8152 - val_loss: 0.3378 - val_acc: 0.8536
Epoch 2/10
625/625 [==============================] - 63s 101ms/step - loss: 0.2375 - acc: 0.9103 - val_loss: 0.3098 - val_acc: 0.8820
Epoch 3/10
625/625 [==============================] - 64s 102ms/step - loss: 0.1851 - acc: 0.9333 - val_loss: 0.2797 - val_acc: 0.8900
Epoch 4/10
625/625 [==============================] - 62s 100ms/step - loss: 0.1513 - acc: 0.9459 - val_loss: 0.3017 - val_acc: 0.8940
Epoch 5/10
625/625 [==============================] - 62s 100ms/step - loss: 0.1298 - acc: 0.9549 - val_loss: 0.3158 - val_acc: 0.8926
Epoch 6/10
625/625 [==============================] - 62s 99ms/step - loss: 0.1136 - acc: 0.9606 - val_loss: 0.3485 - val_acc: 0.8858
Epoch 7/10
625/625 [==============================] - 63s 100ms/step - loss: 0.0951 - acc: 0.9692 - val_loss: 0.3098 - val_acc: 0.8864
Epoch 8/10
625/625 [==============================] - 62

In [18]:
results = model.evaluate(test_data, test_labels)
print('Loss: {}    Accuracy: {}'.format(results[0],results[1]))

782/782 [==============================] - 20s 25ms/step - loss: 0.9683 - acc: 0.7812
Loss: 0.9682536125183105    Accuracy: 0.7811599969863892
